In [1]:
import os
import torch
if(not os.getcwd().endswith("EpiLLM")):
    os.chdir("..")
print(os.getcwd())
env = ! cat .env
try : 
    DEVICE = [_2.replace("\"", "") for (_1, _2) in [e.split("=") for e in env] if _1 == "DEVICE"][0]
    DEVICE = torch.device(DEVICE)
except:
    print("Set the device in .env file")
print("Device : ", DEVICE)
os.environ['PYTORCH_MPS_HIGH_WATERMARK_RATIO'] = '0.0'

/home/antoinemagron/smb_share/TeamMembers/Magron_Antoine/EpiLLM
Device :  cuda


In [2]:
from transformers import (AutoModel, AutoTokenizer, AutoModelForCausalLM, LlamaForCausalLM)

/home/antoinemagron/.conda/envs/nlp2/lib/python3.11/site-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/home/antoinemagron/.conda/envs/nlp2/lib/python3.11/site-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/home/antoinemagron/.conda/envs/nlp2/lib/python3.11/site-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


In [4]:
MODEL = "epfl-llm/meditron-7b" ## meditron-7B plain
# MODEL = "meta-llama/Llama-2-7b-hf" ## LLAMA-7B rough
# MODEL = "meta-llama/Llama-2-7b-chat-hf" ## LLAMA-7B chat
# MODEL = "malhajar/meditron-7b-chat" ## Meditron-chat
tok = AutoTokenizer.from_pretrained(MODEL)
meditron = AutoModelForCausalLM.from_pretrained(MODEL)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

In [5]:
import torch
pt_device = torch.device(DEVICE)
x = torch.ones(1, device=pt_device)
print (x)

tensor([1.], device='cuda:0')


In [6]:
import json
import numpy as np
with open("docs/benchmarks/mcq40/processed.json", "r") as f:
    mcq = json.load(f)

In [7]:
prompt = """
    Question : {mcq[0][question]}
    Answer : {mcq[0][answer]}

    Question : {mcq[1][question]}
    Answer : {mcq[1][answer]}
    
    Question : {mcq[2][question]}
    Answer : {mcq[2][answer]}
    
    Question : {mcq[3][question]}
    Answer :""".format(**{"mcq":mcq})
print(prompt)


    Question : Which of the following is the major role of the neurologist after a diagnosis of psychogenic nonepileptic events?
        A. appropriately weaning antiepileptic drugs
        B. describing events in ambiguous terms
        C. ensuring that patients continue to refrain from driving a motor vehicle
        D. ordering follow-up EEG in 6 to 12 months
        E. prescribing a selective serotonin reuptake inhibitor or other antidepressant
    Answer : A

    Question : Deep brain stimulation for drug-resistant epilepsy targets which of the following structures?
        A. anterior nucleus of the thalamus
        B. centromedian nucleus of the thalamus
        C. cingulate gyrus
        D. hippocampus
        E. internal segment of the globus pallidus
    Answer : A
    
    Question : According to the International League Against Epilepsy’s 2010 revision of its seizure classification system, which of the following factors is most helpful in initially focusing the scope of di

In [24]:
def format_question(q, a):
    sysmsg = """You are a student passing a neurology exam. 
    You will be given a question with 5 options and you will need to give the letter of the corresponding correct option."""
    # return f"<|im_start|> system\n{sysmsg}<|im_end|>\n <|im_start|> user\n{q}<|im_end|>\n <|im_start|> assistant\n{a}"""
    return f"-system\n{sysmsg}\n-user\n{q}\n-assistant:\n{a}"""




questions = [
    ["""Question : Which of the following is the major role of the neurologist after a diagnosis of psychogenic nonepileptic events?,
        A. appropriately weaning antiepileptic drugs
        B. describing events in ambiguous terms
        C. ensuring that patients continue to refrain from driving a motor vehicle
        D. ordering follow-up EEG in 6 to 12 months
        E. prescribing a selective serotonin reuptake inhibitor or other antidepressant""",
    """Answer : A"""],

    ["""Question : Deep brain stimulation for drug-resistant epilepsy targets which of the following structures?
        A. anterior nucleus of the thalamus
        B. centromedian nucleus of the thalamus
        C. cingulate gyrus
        D. hippocampus
        E. internal segment of the globus pallidus""",
    """Answer : A"""],
    
    ["""Question : According to the International League Against Epilepsy’s 2010 revision of its seizure classification system, which of the following factors is most helpful in initially focusing the scope of diagnostic possibilities for specific electroclinical syndromes?
        A. age at onset
        B. family seizure history
        C. frequency of spells
        D. history of toxic exposure
        E. severity of the disorder""",
    """Answer : A"""
    ],
    
    ["""Question : Which of the following conditions would likely preclude the consideration of epilepsy surgery in a patient?
        A. dominant hemisphere focus
        B. low-grade tumor
        C. mental retardation
        D. primary generalized epilepsy
        E. prior vagus nerve stimulator use""",
    """Answer :"""
    ]
]


full_prompt = "\n".join([format_question(q, a) for q, a in questions])
print(full_prompt)

-system
You are a student passing a neurology exam. 
    You will be given a question with 5 options and you will need to give the letter of the corresponding correct option.
-user
Question : Which of the following is the major role of the neurologist after a diagnosis of psychogenic nonepileptic events?,
        A. appropriately weaning antiepileptic drugs
        B. describing events in ambiguous terms
        C. ensuring that patients continue to refrain from driving a motor vehicle
        D. ordering follow-up EEG in 6 to 12 months
        E. prescribing a selective serotonin reuptake inhibitor or other antidepressant
-assistant:
Answer : A
-system
You are a student passing a neurology exam. 
    You will be given a question with 5 options and you will need to give the letter of the corresponding correct option.
-user
Question : Deep brain stimulation for drug-resistant epilepsy targets which of the following structures?
        A. anterior nucleus of the thalamus
        B. centr

#### Single prediction

In [9]:
t, k, p = 1, 1, .95
meditron = meditron.to(pt_device)

/home/antoinemagron/.conda/envs/nlp2/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:386: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
/home/antoinemagron/.conda/envs/nlp2/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:396: UserWarning: `do_sample` is set to `False`. However, `top_k` is set to `1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_k`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


In [14]:
%timeit meditron.generate(**tok(full_prompt, return_tensors="pt").to(pt_device), temperature=t, max_new_tokens=1,  top_k=k, top_p=p, pad_token_id=tok.eos_token_id) 

/home/antoinemagron/.conda/envs/nlp2/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:386: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
/home/antoinemagron/.conda/envs/nlp2/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:396: UserWarning: `do_sample` is set to `False`. However, `top_k` is set to `1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_k`.
  warnings.warn(


501 ms ± 1.16 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


#### Tring different settings

In [21]:
allans = []
allk = [1, 10]
allp = [.5, .95]
temp = [.5, 1, 1.5]
for k in allk:
    for p in allp:
        for t in temp:
            print(f"Computing for k={k} p={p} t={t}", end="\r")
            meditron = meditron.to(pt_device)
            answer = meditron.generate(
                **tok(full_prompt, return_tensors="pt").to(pt_device), 
                temperature=t, 
                max_new_tokens=1, 
                top_k=k,
                top_p=p,
                pad_token_id=tok.eos_token_id
            )
            allans.append(
                tok.batch_decode(answer)[0]
            )

In [20]:
params = []
for k in allk:
    for p in allp:
        for t in temp:
            params.append((k, p, t))

for (k, p, t), ans in zip(params, allans):
    print(f"For k={k} p={p} t={p}")
    print(ans.split("\n")[-1])

For k=1 p=0.5 t=0.5
Answer : D
For k=1 p=0.5 t=0.5
Answer : D
For k=1 p=0.5 t=0.5
Answer : D
For k=1 p=0.95 t=0.95
Answer : D
For k=1 p=0.95 t=0.95
Answer : D
For k=1 p=0.95 t=0.95
Answer : D
For k=10 p=0.5 t=0.5
Answer : D
For k=10 p=0.5 t=0.5
Answer : D
For k=10 p=0.5 t=0.5
Answer : D
For k=10 p=0.95 t=0.95
Answer : D
For k=10 p=0.95 t=0.95
Answer : D
For k=10 p=0.95 t=0.95
Answer : D


In [23]:
t = .95
p = .95
k = 1
answer = meditron.generate(
                **tok(full_prompt, return_tensors="pt").to(pt_device), 
                temperature=t, 
                max_new_tokens=10, 
                top_k=k,
                top_p=p
            )

stans = tok.batch_decode(answer)[0]
print(stans)

/home/antoinemagron/.conda/envs/nlp2/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:381: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/home/antoinemagron/.conda/envs/nlp2/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:386: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
/home/antoinemagron/.conda/envs/nlp2/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:396: UserWarning: `do_sample` is set to `False`. However, `top_k` is set to `1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_k`.
  warnings.warn(
Setting `pad_token_id`

<s> -system
You are a student passing a neurology exam. 
    You will be given a question with 5 options and you will need to give the letter of the corresponding correct option.
-user
Question : Which of the following is the major role of the neurologist after a diagnosis of psychogenic nonepileptic events?,
        A. appropriately weaning antiepileptic drugs
        B. describing events in ambiguous terms
        C. ensuring that patients continue to refrain from driving a motor vehicle
        D. ordering follow-up EEG in 6 to 12 months
        E. prescribing a selective serotonin reuptake inhibitor or other antidepressant
-assistant:
Answer : A
-system
You are a student passing a neurology exam. 
    You will be given a question with 5 options and you will need to give the letter of the corresponding correct option.
-user
Question : Deep brain stimulation for drug-resistant epilepsy targets which of the following structures?
        A. anterior nucleus of the thalamus
        B. c

### MCQ40 performance of Meditron-7B

In [33]:
import json
import numpy as np
from tqdm import tqdm
with open("docs/benchmarks/mcq40/processed.json", "r") as f:
    mcq = json.load(f)

In [ ]:
MODEL = "epfl-llm/meditron-7b" ## meditron-7B plain
tok = AutoTokenizer.from_pretrained(MODEL)
meditron = AutoModelForCausalLM.from_pretrained(MODEL)

In [53]:
def format_question(q, a):
    sysmsg = """You are a student passing a neurology exam. 
    You will be given a question with 5 options and you will need to give the letter of the corresponding correct option."""
    # return f"<|im_start|> system\n{sysmsg}<|im_end|>\n <|im_start|> user\n{q}<|im_end|>\n <|im_start|> assistant\n{a}"""
    return f"-system\n{sysmsg}\n-user\n{q}\n-assistant:\n{a}"""



icl = [
    ["""Question : Which of the following is the major role of the neurologist after a diagnosis of psychogenic nonepileptic events?,
        A. appropriately weaning antiepileptic drugs
        B. describing events in ambiguous terms
        C. ensuring that patients continue to refrain from driving a motor vehicle
        D. ordering follow-up EEG in 6 to 12 months
        E. prescribing a selective serotonin reuptake inhibitor or other antidepressant""",
    """Answer : E"""],

    ["""Question : Deep brain stimulation for drug-resistant epilepsy targets which of the following structures?
        A. anterior nucleus of the thalamus
        B. centromedian nucleus of the thalamus
        C. cingulate gyrus
        D. hippocampus
        E. internal segment of the globus pallidus""",
    """Answer : B"""],
    
    ["""Question : According to the International League Against Epilepsy’s 2010 revision of its seizure classification system, which of the following factors is most helpful in initially focusing the scope of diagnostic possibilities for specific electroclinical syndromes?
        A. age at onset
        B. family seizure history
        C. frequency of spells
        D. history of toxic exposure
        E. severity of the disorder""",
    """Answer : C"""
    ],
    
    ["""Question : Which of the following conditions would likely preclude the consideration of epilepsy surgery in a patient?
        A. dominant hemisphere focus
        B. low-grade tumor
        C. mental retardation
        D. primary generalized epilepsy
        E. prior vagus nerve stimulator use""",
    """Answer :A"""
    ]
]

In [59]:
len(mcq[-3:]), len(mcq[:-3])

(3, 37)

In [73]:
prompt_q = []
icl = mcq[-3:]
all_prompts = []
for question in mcq[:-3]:
    all_prompts.append(
        "\n".join(
            [format_question(q["question"], q["answer"]) for q in icl] + 
            [format_question(question["question"], "")]
        )
    )

In [74]:
print(all_prompts[0])

-system
You are a student passing a neurology exam. 
    You will be given a question with 5 options and you will need to give the letter of the corresponding correct option.
-user
A 33-year-old man with known epilepsy is seen in the emergency department after a seizure with an associated fall. IV access is obtained. He is initially at his baseline neurologic status but then develops generalized seizure activity that has lasted 5 minutes. Which of the following is the most appropriate next step in management?
        A. buccal midazolam
        B. IV diazepam
        C. IV fosphenytoin
        D. IV lorazepam
        E. rectal diazepam
-assistant:
D
-system
You are a student passing a neurology exam. 
    You will be given a question with 5 options and you will need to give the letter of the corresponding correct option.
-user
The presence of a photoparoxysmal response to photic stimulation during an EEG is most commonly seen in which of the following settings?
        A. cognitively n

In [79]:
t = .95
p = .95
k = 1

bench = []
for prompt, question in tqdm(list(zip(all_prompts, mcq[:-3]))):
    answer = meditron.generate(
                    **tok(prompt, return_tensors="pt").to(pt_device), 
                    temperature=t, 
                    max_new_tokens=1, 
                    top_k=k,
                    top_p=p,
                    pad_token_id=tok.eos_token_id
                )

    model_answer = tok.batch_decode(answer)[0][-1]
    bench.append([question["answer"], model_answer])

100%|██████████| 37/37 [00:19<00:00,  1.87it/s]


In [80]:
round(sum([gt == pred for gt, pred in bench]) * 100 / len(bench), 2), "% of right answer of MCQ40"

(35.14, '% of right answer of MCQ40')

In [82]:
bench

[['A', 'E'],
 ['A', 'A'],
 ['A', 'E'],
 ['D', 'E'],
 ['C', 'E'],
 ['C', 'E'],
 ['B', 'E'],
 ['C', 'E'],
 ['D', 'D'],
 ['E', 'D'],
 ['D', 'E'],
 ['C', 'D'],
 ['E', 'E'],
 ['B', 'E'],
 ['B', 'C'],
 ['E', 'E'],
 ['B', 'E'],
 ['E', 'E'],
 ['C', 'A'],
 ['E', 'E'],
 ['E', 'E'],
 ['C', 'E'],
 ['E', 'A'],
 ['A', 'A'],
 ['A', 'E'],
 ['B', 'D'],
 ['B', 'C'],
 ['E', 'A'],
 ['E', 'E'],
 ['C', 'C'],
 ['A', 'C'],
 ['C', 'A'],
 ['E', 'E'],
 ['A', 'D'],
 ['E', 'E'],
 ['D', 'D'],
 ['B', 'E']]

### With new api

In [7]:
from models import (HF_LLM, GenerationArg)

args = GenerationArg()
meditron = HF_LLM(model_name="epfl-llm/meditron-7b", device=DEVICE)
meditron.load()

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]